Fine-tuning best T5 Transformer 🤖
-----------------------------------

In this notebook, we will continue the fine-tuning of T5 transformer on the new extracted sentences from the book **Grammaire de Wolof Moderne** without considering the definitions. We provide, bellow, the main evaluation figures, obtained from the hyperparameter search step. We will evaluate the training on the validation dataset.

- Parallel coordinates from panel:

- Parameter importance char: 
[t5_v3_importance](https://wandb.ai/oumar-kane-team/small-t5-cross-fw-translation-bayes-hpsearch-v3/reports/undefined-23-05-16-10-36-17---Vmlldzo0Mzc4NDY0?accessToken=eyaiyrid0qz1zg2jkq3fc65biw53084dpfitbi0dgonq6mweupw6kgjml9d2nv1w)

We can see in the above chart that the batch is the most important parameter with a negative correlation with the BLEU score (meaning that a lower batch size is better). Next, we the probability of modifying a character in the french corpus is also important and a high probability provide a better BLEU score.  

In [1]:
# let us import all necessary libraries
from transformers import AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer, T5TokenizerFast, set_seed, AdamW, get_linear_schedule_with_warmup, T5ForConditionalGeneration,\
    get_cosine_schedule_with_warmup, Adafactor
from wolof_translate.utils.sent_transformers import TransformerSequences
from wolof_translate.utils.improvements.end_marks import add_end_mark # added
from torch.nn import TransformerEncoderLayer, TransformerDecoderLayer
from torch.utils.data import Dataset, DataLoader, random_split
from wolof_translate.data.dataset_v4 import SentenceDataset # v2 -> v3 -> v4
from wolof_translate.utils.sent_corrections import *
from sklearn.model_selection import train_test_split
from torch.optim.lr_scheduler import _LRScheduler
# from custom_rnn.utils.kwargs import Kwargs
from torch.nn.utils.rnn import pad_sequence
from plotly.subplots import make_subplots
from nlpaug.augmenter import char as nac
from torch.utils.data import DataLoader
# from datasets  import load_metric # make pip install evaluate instead
# and pip install sacrebleu for instance
from torch.nn import functional as F
import plotly.graph_objects as go
from tokenizers import Tokenizer
import matplotlib.pyplot as plt
import pytorch_lightning as lt
from tqdm import tqdm, trange
from functools import partial
from torch.nn import utils
from copy import deepcopy
from torch import optim
from typing import *
from torch import nn
import pandas as pd
import numpy as np
import itertools
import evaluate
import random
import string
import shutil
import wandb
import torch
import json
import copy
import os

# add seed for everything
lt.seed_everything(0)

os.environ["WANDB_DISABLED"] = "true"

c:\Users\Oumar Kane\AppData\Local\pypoetry\Cache\virtualenvs\pytorch1-HleOW5am-py3.10\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Global seed set to 0


## French to wolof

### Configure dataset 🔠

In [2]:
# recuperate the tokenizer from a json file
tokenizer = T5TokenizerFast(tokenizer_file=f"wolof-translate/wolof_translate/tokenizers/t5_tokenizers/tokenizer_v3_2.json")

In [3]:
def recuperate_datasets(fr_char_p: float, fr_word_p: float, max_len: int, end_mark_opt: int):

  # Let us recuperate the end_mark adding option
  if end_mark_opt == 1:
    # Create augmentation to add on French sentences
    fr_augmentation_1 = TransformerSequences(nac.KeyboardAug(aug_char_p=fr_char_p, aug_word_p=fr_word_p,
                                                             aug_word_max = max_len),
                                          remove_mark_space, delete_guillemet_space, add_mark_space)

    fr_augmentation_2 = TransformerSequences(remove_mark_space, delete_guillemet_space, add_mark_space)
    
  else:
    
    if end_mark_opt == 2:

      end_mark_fn = partial(add_end_mark, end_mark_to_remove = '!', replace = True)
    
    elif end_mark_opt == 3:

      end_mark_fn = partial(add_end_mark)
    
    elif end_mark_opt == 4:

      end_mark_fn = partial(add_end_mark, end_mark_to_remove = '!')

    # Create augmentation to add on French sentences
    fr_augmentation_1 = TransformerSequences(nac.KeyboardAug(aug_char_p=fr_char_p, aug_word_p=fr_word_p,
                                                             aug_word_max = max_len),
                                          remove_mark_space, delete_guillemet_space, add_mark_space, end_mark_fn)
    
    fr_augmentation_2 = TransformerSequences(remove_mark_space, delete_guillemet_space, add_mark_space, end_mark_fn)
    
  # Recuperate the train dataset
  train_dataset_aug = SentenceDataset(f"data/extractions/new_data/train_set.csv",
                                        tokenizer,
                                        truncation = True, max_len=max_len,
                                        cp1_transformer = fr_augmentation_1,
                                        cp2_transformer = fr_augmentation_2
                                        )

  # Recuperate the valid dataset
  valid_dataset = SentenceDataset(f"data/extractions/new_data/valid_set.csv",
                                        tokenizer, max_len=max_len,
                                        cp1_transformer = fr_augmentation_2,
                                        cp2_transformer = fr_augmentation_2,
                                        truncation = True)
  
  # Return the datasets
  return train_dataset_aug, valid_dataset

### Configure the model and the evaluation function ⚙️

Let us evaluate the predictions with the `bleu` metric.

Let us initialize the evaluation object.

In [4]:
%run wolof-translate/wolof_translate/utils/evaluation.py
evaluation = TranslationEvaluation(tokenizer)


### Searching for the best parameters 🕖

In [5]:
from wolof_translate.models.transformers.optimization import TransformerScheduler
from wolof_translate.trainers.transformer_trainer import ModelRunner
from wolof_translate.utils.evaluation import TranslationEvaluation
from wolof_translate.models.transformers.main import Transformer
from wolof_translate.utils.split_with_valid import split_data


-------------

### --- Wandb v5 2000

In [6]:
# let us initialize the hyperparameter configuration 
config = {
    'random_state': 0,
    'fr_char_p': 0.23724577978551456,
    'fr_word_p': 0.08738178822117504,
    'learning_rate': 0.0010289893593708468,
    'weight_decay': 0.017712187484021313,
    'batch_size': 16,
    'warmup_ratio': 0.0,
    'max_epoch': 404,
    'epochs': 40,
    'mid_epoch': 40,
    'max_len': 17,
    'end_mark': 3,
    'bleu': 7.0873,
    'model_dir': 'data/checkpoints/fw_t5_small_custom_train_v5_checkpoints/',
    'new_model_dir': 'data/checkpoints/t5_small_custom_train_results_fw_v5/'
}

# Initialize the model name
model_name = 't5-small'

# import the model with its pre-trained weights
model = T5ForConditionalGeneration.from_pretrained(model_name)

# resize the token embeddings
model.resize_token_embeddings(len(tokenizer))

# let us initialize the evaluation class
evaluation = TranslationEvaluation(tokenizer)

# let us initialize the trainer
trainer = ModelRunner(model, seed = 0, version = 5, evaluation = evaluation, optimizer=Adafactor)

# split the data
split_data(config['random_state'], csv_file = "ad_sentences.csv")

# recuperate train and test set
train_dataset, test_dataset = recuperate_datasets(config['fr_char_p'], 
                                                    config['fr_word_p'], config['max_len'],
                                                    config['end_mark'])

# let us calculate the appropriate warmup steps (let us take a max epoch of 100)
length = len(train_dataset)

n_steps = length // config['batch_size']

num_steps = config['max_epoch'] * n_steps

warmup_steps = (config['max_epoch'] * n_steps) * config['warmup_ratio']

# # Initialize the scheduler parameters
scheduler_args = {'num_warmup_steps': warmup_steps, 'num_training_steps': num_steps}

# Initialize the optimizer parameters
optimizer_args = {
    'lr': config['learning_rate'],
    'weight_decay': config['weight_decay'],
    # 'betas': (0.9, 0.98),
    'warmup_init': False,
    'relative_step': False
}

# Initialize the loaders parameters
train_loader_args = {'batch_size': config['batch_size']}

# Add the datasets and hyperparameters to trainer
trainer.compile(train_dataset, test_dataset, tokenizer, train_loader_args,
                optimizer_kwargs = optimizer_args,
                lr_scheduler=get_linear_schedule_with_warmup,
                lr_scheduler_kwargs=scheduler_args, 
                predict_with_generate = True,
                hugging_face = True,
                logging_dir="data/logs/t5_small_custom_train_fw"
                )

# We will from checkpoints so let us the model
trainer.load(config['model_dir'], load_best=True) # Only for the first loading
# trainer.load(config['new_model_dir'])

        

### --- Linear

In [11]:
trainer.train(epochs = config['epochs'] - trainer.current_epoch, auto_save=True, metric_for_best_model='bleu', metric_objective='maximize', log_step=1,
              saving_directory = config['new_model_dir'])

  0%|          | 0/35 [00:00<?, ?it/s]c:\Users\Oumar Kane\AppData\Local\pypoetry\Cache\virtualenvs\pytorch1-HleOW5am-py3.10\lib\site-packages\torch\optim\lr_scheduler.py:257: UserWarning: To get the last learning rate computed by the scheduler, please use `get_last_lr()`.
  warnings.warn("To get the last learning rate computed by the scheduler, "


For epoch 6: 
{Learning rate: [0.001016124393395686]}


Test batch number 13: 100%|██████████| 13/13 [00:03<00:00,  3.27batches/s]



Metrics: {'train_loss': 1.6284285639271592, 'test_loss': 1.9188752632874708, 'bleu': 5.7887, 'gen_len': 6.8838}




  3%|▎         | 1/35 [00:32<18:32, 32.72s/it]

...

 97%|█████████▋| 34/35 [16:33<00:28, 28.40s/it]

For epoch 40: 
{Learning rate: [0.0009286426247645921]}


Test batch number 13: 100%|██████████| 13/13 [00:03<00:00,  3.36batches/s]



Metrics: {'train_loss': 0.13329573223988214, 'test_loss': 2.3199255283062277, 'bleu': 17.5266, 'gen_len': 8.3182}




100%|██████████| 35/35 [17:02<00:00, 29.21s/it]


### --- Cosine

In [ ]:
trainer.train(epochs = config['epochs'] - trainer.current_epoch, auto_save=True, metric_for_best_model='bleu', metric_objective='maximize', log_step=1,
              saving_directory = config['new_model_dir'])

### --- Wandb v5_2 3000

127

In [7]:
# let us initialize the hyperparameter configuration 
config = {
    'random_state': 0,
    'fr_char_p': 0.06673707344553906,
    'fr_word_p': 0.07626063714036732,
    'learning_rate': 0.0030232985434635257,
    'weight_decay': 0.02659316315720403,
    'batch_size': 16,
    'warmup_ratio': 0.0,
    'max_epoch': 827,
    'epochs': 800,
    'mid_epoch': 83,
    'max_len': 19,
    'end_mark': 3,
    'bleu': 8.4915,
    'model_dir': 'data/checkpoints/fw_t5_small_custom_train_v5_2_checkpoints',
    'new_model_dir': 'data/checkpoints/t5_small_custom_train_results_fw_v5_2'
}

# Initialize the model name
model_name = 't5-small'

# import the model with its pre-trained weights
model = T5ForConditionalGeneration.from_pretrained(model_name)

# resize the token embeddings
model.resize_token_embeddings(len(tokenizer))

# let us initialize the evaluation class
evaluation = TranslationEvaluation(tokenizer)

# let us initialize the trainer
trainer = ModelRunner(model, seed = 0, version = '5_2', evaluation = evaluation, optimizer=Adafactor)

# split the data
split_data(config['random_state'], csv_file = "ad_sentences.csv")

# recuperate train and test set
train_dataset, test_dataset = recuperate_datasets(config['fr_char_p'], 
                                                    config['fr_word_p'], config['max_len'],
                                                    config['end_mark'])

# let us calculate the appropriate warmup steps (let us take a max epoch of 100)
length = len(train_dataset)

n_steps = length // config['batch_size']

num_steps = config['max_epoch'] * n_steps

warmup_steps = (config['max_epoch'] * n_steps) * config['warmup_ratio']

# # Initialize the scheduler parameters
scheduler_args = {'num_warmup_steps': warmup_steps, 'num_training_steps': num_steps}

# Initialize the optimizer parameters
optimizer_args = {
    'lr': config['learning_rate'],
    'weight_decay': config['weight_decay'],
    # 'betas': (0.9, 0.98),
    'warmup_init': False,
    'relative_step': False
}

# Initialize the loaders parameters
train_loader_args = {'batch_size': config['batch_size']}

# Add the datasets and hyperparameters to trainer
trainer.compile(train_dataset, test_dataset, tokenizer, train_loader_args,
                optimizer_kwargs = optimizer_args,
                lr_scheduler=get_cosine_schedule_with_warmup,
                lr_scheduler_kwargs=scheduler_args, 
                predict_with_generate = True,
                hugging_face = True,
                logging_dir="data/logs/t5_small_custom_train_fw"
                )

# We will from checkpoints so let us the model
# trainer.load(config['model_dir'], load_best=True) # Only for the first loading
trainer.load(config['new_model_dir'], load_best = True)

        

### --- Linear step

In [7]:
trainer.train(epochs = config['epochs'] - trainer.current_epoch, auto_save=True, metric_for_best_model='bleu', metric_objective='maximize', log_step=1,
              saving_directory = config['new_model_dir'])

  0%|          | 0/78 [00:00<?, ?it/s]/home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/torch/optim/lr_scheduler.py:265: UserWarning: To get the last learning rate computed by the scheduler, please use `get_last_lr()`.
  warnings.warn("To get the last learning rate computed by the scheduler, "


For epoch 6: 
{Learning rate: [0.0030048333165675106]}


Test batch number 13: 100%|██████████| 13/13 [00:11<00:00,  1.13batches/s]



Metrics: {'train_loss': 1.0044883995345144, 'test_loss': 1.7446289887795081, 'bleu': 7.0496, 'gen_len': 8.6515}




  1%|▏         | 1/78 [00:41<53:27, 41.65s/it]

...

 99%|█████████▊| 77/78 [24:08<00:18, 18.25s/it]

For epoch 83: 
{Learning rate: [0.002720468822368877]}


Test batch number 13: 100%|██████████| 13/13 [00:03<00:00,  3.61batches/s]



Metrics: {'train_loss': 0.01930094945404415, 'test_loss': 2.118147923396184, 'bleu': 20.065, 'gen_len': 10.0}




100%|██████████| 78/78 [24:27<00:00, 18.81s/it]


### --- Cosine Step

In [10]:
trainer.train(epochs = config['epochs'] - trainer.current_epoch, auto_save=True, metric_for_best_model='bleu', metric_objective='maximize', log_step=1,
              saving_directory = config['new_model_dir'])

  0%|          | 0/217 [00:00<?, ?it/s]/home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/torch/optim/lr_scheduler.py:265: UserWarning: To get the last learning rate computed by the scheduler, please use `get_last_lr()`.
  warnings.warn("To get the last learning rate computed by the scheduler, "


For epoch 84: 
{Learning rate: [0.002947264271584649]}


Test batch number 13: 100%|██████████| 13/13 [00:03<00:00,  3.78batches/s]



Metrics: {'train_loss': 0.02082285217263482, 'test_loss': 2.092761048903832, 'bleu': 20.2433, 'gen_len': 9.9697}




  0%|          | 1/217 [00:18<1:05:15, 18.13s/it]

...

100%|█████████▉| 216/217 [1:05:35<00:17, 17.94s/it]

For epoch 300: 
{Learning rate: [0.0021326885745470395]}


Test batch number 13: 100%|██████████| 13/13 [00:03<00:00,  3.94batches/s]



Metrics: {'train_loss': 0.007315988333147922, 'test_loss': 2.009385576614967, 'bleu': 21.0837, 'gen_len': 10.0859}




100%|██████████| 217/217 [1:05:53<00:00, 18.22s/it]


### ---

In [ ]:
trainer.train(epochs = config['epochs'] - trainer.current_epoch, auto_save=True, metric_for_best_model='bleu', metric_objective='maximize', log_step=1,
              saving_directory = config['new_model_dir'])

  0%|          | 0/200 [00:00<?, ?it/s]/home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/torch/optim/lr_scheduler.py:265: UserWarning: To get the last learning rate computed by the scheduler, please use `get_last_lr()`.
  warnings.warn("To get the last learning rate computed by the scheduler, "


For epoch 301: 
{Learning rate: [0.0021273951678847957]}


Test batch number 13: 100%|██████████| 13/13 [00:03<00:00,  3.89batches/s]



Metrics: {'train_loss': 0.007536539225692089, 'test_loss': 2.0039380330305834, 'bleu': 20.6829, 'gen_len': 9.8889}




  0%|          | 1/200 [00:17<58:53, 17.76s/it]

...

Test batch number 13: 100%|██████████| 13/13 [00:03<00:00,  3.78batches/s]



Metrics: {'train_loss': 0.00633435653236864, 'test_loss': 2.036895201756404, 'bleu': 22.4482, 'gen_len': 9.9293}




 68%|██████▊   | 137/200 [41:33<19:26, 18.51s/it]

For epoch 438: 
{Learning rate: [0.0013513974206517299]}


Test batch number 9:  62%|██████▏   | 8/13 [00:02<00:01,  3.58batches/s]

### ---

In [7]:
trainer.train(epochs = config['epochs'] - trainer.current_epoch, auto_save=True, metric_for_best_model='bleu', metric_objective='maximize', log_step=1,
              saving_directory = config['new_model_dir'])

  0%|          | 0/60 [00:00<?, ?it/s]/home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/torch/optim/lr_scheduler.py:265: UserWarning: To get the last learning rate computed by the scheduler, please use `get_last_lr()`.
  warnings.warn("To get the last learning rate computed by the scheduler, "


For epoch 441: 
{Learning rate: [0.0013341034239208402]}


Test batch number 13: 100%|██████████| 13/13 [00:03<00:00,  3.66batches/s]



Metrics: {'train_loss': 0.006328606113175287, 'test_loss': 2.043260097503662, 'bleu': 21.8211, 'gen_len': 9.6616}




  2%|▏         | 1/60 [00:21<21:27, 21.82s/it]

...

 98%|█████████▊| 59/60 [20:22<00:19, 19.96s/it]

For epoch 500: 
{Learning rate: [0.001001640061669694]}


Test batch number 13: 100%|██████████| 13/13 [00:03<00:00,  3.93batches/s]



Metrics: {'train_loss': 0.0058785396937640726, 'test_loss': 2.043622686312749, 'bleu': 22.5215, 'gen_len': 9.8384}




100%|██████████| 60/60 [20:41<00:00, 20.69s/it]


### ---

In [10]:
trainer.train(epochs = config['epochs'] - trainer.current_epoch, auto_save=True, metric_for_best_model='bleu', metric_objective='maximize', log_step=1,
              saving_directory = config['new_model_dir'])

  0%|          | 0/344 [00:00<?, ?it/s]/home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/torch/optim/lr_scheduler.py:265: UserWarning: To get the last learning rate computed by the scheduler, please use `get_last_lr()`.
  warnings.warn("To get the last learning rate computed by the scheduler, "


For epoch 457: 
{Learning rate: [0.001242321968914977]}


Test batch number 13: 100%|██████████| 13/13 [00:11<00:00,  1.10batches/s]



Metrics: {'train_loss': 0.007280127252945958, 'test_loss': 2.033146463907682, 'bleu': 21.7641, 'gen_len': 10.0202}




  0%|          | 1/344 [00:27<2:36:57, 27.46s/it]

...

100%|█████████▉| 343/344 [1:41:07<00:17, 17.57s/it]

For epoch 800: 
{Learning rate: [4.2942873224277255e-06]}


Test batch number 13: 100%|██████████| 13/13 [00:03<00:00,  3.80batches/s]



Metrics: {'train_loss': 0.005281617864451931, 'test_loss': 2.086399977023785, 'bleu': 22.5451, 'gen_len': 9.904}




100%|██████████| 344/344 [1:41:25<00:00, 17.69s/it]


### Predictions and Evaluation

In [7]:
# initialize the transformation sequence
end_mark_fn = partial(add_end_mark)
fr_augmentation = TransformerSequences(remove_mark_space, delete_guillemet_space, add_mark_space, end_mark_fn)

# let us get the test set
test_dataset = SentenceDataset(f"data/extractions/new_data/test_set.csv",
                                        tokenizer = tokenizer,
                                        cp1_transformer = fr_augmentation,
                                        cp2_transformer = fr_augmentation,
                                        truncation = True)

Let us make the evaluation and print the predicted sentences.

In [8]:
# evaluation with test set
df_ft_to_wf = trainer.evaluate(test_dataset)

Evaluation batch number 13: 100%|██████████| 13/13 [00:26<00:00,  2.00s/batches]


predictions_: [[0, 249, 38, 75, 3, 6, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 311, 38, 75, 3, 6, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 582, 22, 3, 6, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 356, 15, 5, 48, 5, 4, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 97, 9, 21, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 97, 5, 215, 5, 4, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 97, 9, 21, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 320, 187, 75, 4, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 350, 22, 4, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 97, 129, 78, 4, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 51, 35, 107, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 97, 5, 13, 5, 16, 5, 13, 5, 13, 5, 13, 5, 13, 5, 13, 5, 13, 5, 13], [0, 40, 41, 31, 22, 18, 4, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 249, 3, 6, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 97, 129, 4, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

In [9]:
df_ft_to_wf[1].tail(10)

,original_sentences,translations,predictions
188,Donne le travail à un autre!,Joxal kenn liggéey bi!,Nit la!
189,Cet homme qui a été.,Góor gii demoon.,Góor gi dem.
190,Dites-lui.,Nileen ka.,Góor gi dem.
191,Il est là.,Ma ŋgoogule foofu.,Dem na.
192,Tu vois cet homme là-bas?,Gis ŋga nit kale?,Ma ŋga nit?
193,L'homme est parti je crois!,Ma defe góor gi dem na!,Góor gi dem na!
194,J'ai aperçu un baobab.,Séen naa ag guy.,Dem naa ci biir.
195,Celui-ci serait parti.,Kii dafa demkoon.,Góor gi bëgg na.
196,Toutes les portes étaient ouvertes.,Bunt yi yépp a tëjju woon.,Góor gi di dem.
197,"C'est Fatim, aujourd'hui.","Faatim la, tay.",Maa di dem.


In [10]:
# let us display 100 samples
pd.options.display.max_rows = 100
df_ft_to_wf[1].sample(100)

,original_sentences,translations,predictions
18,Afin que parte qui?,Ndax kan dem?,Ndax dem?
168,Tu as vu les autres amis?,Gis ŋga sa yeneen xarit yi?,Gis ŋga nit?
63,Quelles femmes se sont égarées?,Jigéen ñan ñoo réer?,Yan nit?
175,C'est ce que tu as dit que la jeune femme préf...,Li ŋga wax la ndaw si taamu.,Nit ki ñëw na.
71,Moi-même je n'ai pas été.,Man mii demuma.,Man dem.
86,Ils partent.,Dem nañu.,Dem na.
5,"Les gens n'étaient pas, habituellement, bavards.",Nit ñi daawuñu coow.,"Nit, leen,."
118,Bonne soirée.,Naka ngon si.,Nit ki.
12,Tout autre mouton que tu vois.,Meneen xar mépp moo gis.,Góor gi dem na.
150,Une photo datant d'il y a soixante et neuf ans...,Nataalu juróom benn fukk ak juróom ñeent day w...,"Góor gi bëgg na, bëgg na, bëgg na, bëgg na, bë..."
